In [1]:
!pip install openrouteservice

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import folium
from folium.plugins import BeautifyIcon
import pandas as pd
import openrouteservice as ors

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import math

## Preparación de la base de datos

Cargar bases de datos

In [125]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
#Información sobre pedidos
df = pd.read_excel('/content/drive/Shareddrives/Optimización determinista/Reto/DatosGarcia.xlsx',parse_dates=['Fecha venta', 'Fecha enrutada'])
#df = pd.read_excel('/content/drive/Shareddrives/Optimización determinista/Reto/SNG-21_enero.xlsx', parse_dates=['Fecha venta', 'Fecha enrutada'])
df = df.drop([0],axis=0) # Bodega

In [112]:
#Características del producto
vol = pd.read_excel('/content/drive/Shareddrives/Optimización determinista/Reto/Copia de Sku_DCF_Volumen.xlsx')

Añadir volumen de cada paquete

In [128]:
#Relación entre bases de datos a través de DCF
d =  dict([(i,a) for i,a in zip(vol['DCF'], vol['Vol'])])
volumen = []
for i in range(len(df['DCF'])):
  if df.iloc[i,df.columns.to_list().index('DCF')] in d:
    volumen.append(float(d[df.iloc[i,df.columns.to_list().index('DCF')]]))
  else:
    volumen.append(np.NaN)

df['Volumen'] = volumen
df = df.fillna(df.mean())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  # This is added back by InteractiveShellApp.init_path()


Agrupar direcciones iguales 

In [129]:
#Diccionario con mismos domicilios y el volumen total de sus pedidos
dir_vol = df.groupby(['Dirección']).agg({"Volumen": sum}).to_dict()['Volumen']

In [130]:
cant = []
for i in list(dir_vol.keys()):
  ind = df.loc[df['Dirección'] == i].index.tolist()
  df.loc[ind,'Volumen'] = dir_vol[i]
  cant.append(df.loc[ind,'Cantidad'].sum())
df = df.drop_duplicates(subset=['Dirección'])

Base de datos final

In [180]:
# Modelo de clasificación kMeans
y_pred = []
X_3 = df[['Latitud','Longitud']]
req_date = pd.to_datetime("21/1/2022 23:59:59")

for i in range(1,7):
  kmeans = KMeans(n_clusters=i)
  y_pred.append(kmeans.fit_predict(X_3))

#Base de datos final
X_3.insert(2, 'Volumen',df.Volumen)
X_3.insert(3, 'Cluster',y_pred[0])
X_3.insert(4, 'Open_From', df['Fecha venta'])
X_3.insert(5, 'Open_To', df['Fecha enrutada'])
X_3.insert(6, 'Needed_Amount', cant)
X_3 = X_3.rename(columns={'Latitud': 'Lat', 'Longitud': 'Lon'})
X_3 = X_3.reset_index(drop=True)

X_1 = X_3.copy()
X_1['Cluster'] = y_pred[0]

X_2 = X_3.copy()
X_2['Cluster'] = y_pred[1]

X_3 = X_3.copy()
X_3['Cluster'] = y_pred[2]

X_4 = X_3.copy()
X_4['Cluster'] = y_pred[3]

X_5 = X_3.copy()
X_5['Cluster'] = y_pred[4]

X_6 = X_3.copy()
X_6['Cluster'] = y_pred[5]

Volumenes

In [232]:
X_1.groupby(by = 'Cluster').sum()['Volumen']

Cluster
0    33.34286
Name: Volumen, dtype: float64

In [238]:
pd.DataFrame(X_2.groupby(by = 'Cluster').sum()['Volumen'])

,Volumen
Cluster,
0,14.71657
1,18.62629


In [98]:
X_3.groupby(by = 'Cluster').sum()['Volumen']

Cluster
0    18.62629
1    10.28357
2     3.25000
Name: Volumen, dtype: float64

In [99]:
X_4.groupby(by = 'Cluster').sum()['Volumen']

Cluster
0     6.729441
1    10.283570
2     3.250000
3    11.896849
Name: Volumen, dtype: float64

In [100]:
X_5.groupby(by = 'Cluster').sum()['Volumen']

Cluster
0     6.729441
1     0.849534
2     3.250000
3    11.896849
4     9.434036
Name: Volumen, dtype: float64

In [101]:
X_6.groupby(by = 'Cluster').sum()['Volumen']

Cluster
0    8.119388
1    0.849534
2    3.250000
3    6.672315
4    9.434036
5    3.834587
Name: Volumen, dtype: float64

# Modelo de enrutamiento

## Funciones

Función para mostrar un mapa

In [132]:
#Todos los vehículos están ubicados en CEDIS Mty
depot = [25.70285022561094, -100.32241533001529]

def mapa(X):
  # Centrar mapa para CEDIS Monterrey
  m = folium.Map(location=[25.70285022561094, -100.32241533001529], tiles='cartodbpositron', zoom_start=12)

  # Plot the locations on the map with more info in the ToolTip
  for location in X.itertuples():
      tooltip = folium.map.Tooltip("<h4><b>ID {}</b></p><p>Supplies needed: <b>{}</b></p>".format(
          location.Index, location.Needed_Amount
      ))

      folium.Marker(
          location=[location.Lat, location.Lon],
          tooltip=tooltip,
          icon=BeautifyIcon(
              icon_shape='marker',
              number=int(location.Index),
              spin=True,
              text_color='firebrick',
              background_color="#FFF",
              inner_icon_style="font-size:12px;padding-top:-5px;"
          )
      ).add_to(m)

  # Todos los vehículos están ubicados en CEDIS Mty
  #depot = [25.75, -100.283]

  folium.Marker(
      location=depot,
      icon=folium.Icon(color="green", icon="bus", prefix='fa'),
  #    setZIndexOffset=1000
  ).add_to(m)

  return m

Para establecer una ruta

In [133]:
def ruta(X1):
  r1 = mapa(X1)
  # Define the vehicles
  # https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Vehicle
  vehicles = list()
  vehicles.append(
        ors.optimization.Vehicle(
            id=0,
             start=list(reversed(depot)),
             end=list(reversed(depot)),
            capacity=[300],
            time_window=[1642744800, 1642795200] # fecha 1-21-2022 0:00 a las 1-21-2022 14:00
        )
    )
  
#segundo vehículo (características de no uso, es solo para que funcione la api porque necesita más de uno)
  vehicles.append(
        ors.optimization.Vehicle(
            id=1,
            start=list(reversed(depot)),
            end=list(reversed(depot)),
            capacity=[1],
            time_window=[1642834800, 1642881600] # fecha 1-22-2022 1:00 a las 1-22-2022 14:00
        )
    )


  # Next define the delivery stations
  # https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Job
  deliveries = list()
  for delivery in X1.itertuples():
      deliveries.append(
          ors.optimization.Job(
              id=delivery.Index,
              location=[delivery.Lon, delivery.Lat],
              service=600,  # Assume 1 minute at each site
              amount=[delivery.Needed_Amount],
              time_windows=[[
                  int(delivery.Open_From.timestamp()),  # VROOM expects UNIX timestamp
                  int(delivery.Open_To.timestamp())
              ]]
          )
      )

  # Initialize a client and make the request
  ors_client = ors.Client(key='5b3ce3597851110001cf6248ae85c9e487f7465586dc4dfe4ad26ff2')  # Get an API key from https://openrouteservice.org/dev/#/signup
  result = ors_client.optimization(
      jobs=deliveries,
      vehicles=vehicles,
      geometry=True
  )

  # Add the output to the map
  for color, route in zip(['goldenrod','slategray','royalblue'], result['routes']):
      decoded = ors.convert.decode_polyline(route['geometry'])  # Route geometry is encoded
      gj = folium.GeoJson(
          name='Vehicle {}'.format(route['vehicle']),
          data={"type": "FeatureCollection", "features": [{"type": "Feature",
                                                          "geometry": decoded,
                                                          "properties": {"color": color}
                                                          }]},
          style_function=lambda x: {"color": x['properties']['color']}
      )
      gj.add_child(folium.Tooltip(
          """<h4>Vehicle {vehicle}</h4>
          <b>Distance</b> {distance} m <br>
          <b>Duration</b> {duration} secs
          """.format(**route)
      ))
      gj.add_to(r1)

  folium.LayerControl().add_to(r1)

  return r1, resumen(result), recorrido(result)

Para obtener el resumen de la ruta

In [134]:
def resumen(result):
  # Only extract relevant fields from the response
  extract_fields = ['distance', 'amount', 'duration']
  data = [{key: route[key] for key in extract_fields} for route in result['routes']]

  vehicles_df = pd.DataFrame(data)
  vehicles_df.index.name = 'vehicle'
  return vehicles_df

Para obtener el recorrido de la ruta con los tiempos

In [135]:
def recorrido(result):
  # Create a list to display the schedule for all vehicles
  stations = list()
  for route in result['routes']:
      vehicle = list()
      for step in route["steps"]:
          vehicle.append(
              [
                  step.get("job", "Depot"),  # Station ID
                  step["arrival"],  # Arrival time
                  step["arrival"] + step.get("service", 0),  # Departure time
              ]
          )
      stations.append(vehicle)

  df_stations_0 = pd.DataFrame()
  for i in range(len(result['routes'])):
    df_stations_0 = pd.concat([df_stations_0, pd.DataFrame(stations[i])])

  #df_stations_0 = pd.DataFrame(stations[0], columns=["Station ID", "Arrival", "Departure"])
  df_stations_0.columns=["Station ID", "Arrival", "Departure"]
  df_stations_0['Arrival'] = pd.to_datetime(df_stations_0['Arrival'], unit='s')
  df_stations_0['Departure'] = pd.to_datetime(df_stations_0['Departure'], unit='s')
  return df_stations_0

## Resultados individuales

### K = 1

In [181]:
#Datos respecto al cluster en que se encuentran
X_11 = X_1[X_1.Cluster == 0]

In [182]:
# Ruta 1
mapa_11, resumen_11, recorrido_11 = ruta(X_11)
resumen_11['tiempo_total (hrs)'] = recorrido_11.Arrival[recorrido_11.shape[0]-1] - recorrido_11.Arrival[0]
resumen_1 = pd.concat([resumen_11], keys=['1'], names = ['ruta','vehicle'])

In [187]:
res_general = {}
res_general[1] = {'distancia': resumen_1.groupby(level = 'ruta').sum().sum()[0], 'duracion':resumen_1.groupby(level = 'ruta').sum().sum()[1], 'dur total':resumen_1['tiempo_total (hrs)'].sum()}

### K = 2


In [188]:
#Datos respecto al cluster en que se encuentran
X_21 = X_2[X_2.Cluster == 0]
X_22 = X_2[X_2.Cluster == 1]

In [189]:
# Ruta 1
mapa_21, resumen_21, recorrido_21 = ruta(X_21)

In [236]:
mapa_21

In [190]:
# Ruta 2
mapa_22, resumen_22, recorrido_22 = ruta(X_22)

In [237]:
mapa_22

In [192]:
resumen_21['tiempo_total (hrs)'] = recorrido_21.Arrival[recorrido_21.shape[0]-1] - recorrido_21.Arrival[0]
resumen_22['tiempo_total (hrs)'] = recorrido_22.Arrival[recorrido_22.shape[0]-1] - recorrido_22.Arrival[0]
resumen_2 = pd.concat([resumen_21, resumen_22], keys=['1', '2'], names = ['ruta','vehicle'])
res_general[2] = {'distancia': resumen_2.groupby(level = 'ruta').sum().sum()[0], 'duracion':resumen_2.groupby(level = 'ruta').sum().sum()[1], 'dur total':resumen_2['tiempo_total (hrs)'].sum()}

In [235]:
resumen_2

,,distance,amount,duration,tiempo_total (hrs)
ruta,vehicle,,,,
1,0,93498,[29],8910,0 days 05:28:30
2,0,99343,[36],9006,0 days 07:00:06


### K = 3

In [193]:
#Datos respecto al cluster en que se encuentran
X_31 = X_3[X_3.Cluster == 0]
X_32 = X_3[X_3.Cluster == 1]
X_33 = X_3[X_3.Cluster == 2]

In [194]:
# Ruta 1
mapa_31, resumen_31, recorrido_31 = ruta(X_31)

In [195]:
# Ruta 2
mapa_32, resumen_32, recorrido_32 = ruta(X_32)

In [196]:
# Ruta 3
mapa_33, resumen_33, recorrido_33 = ruta(X_33)

Duración total de la ruta (con tiempo de espera)

In [197]:
resumen_31['tiempo_total (hrs)'] = recorrido_31.Arrival[recorrido_31.shape[0]-1] - recorrido_31.Arrival[0]
resumen_32['tiempo_total (hrs)'] = recorrido_32.Arrival[recorrido_32.shape[0]-1] - recorrido_32.Arrival[0]
resumen_33['tiempo_total (hrs)'] = recorrido_33.Arrival[recorrido_33.shape[0]-1] - recorrido_33.Arrival[0]

Distancia, paquetes entregados y duración de cada ruta

In [198]:
resumen_3 = pd.concat([resumen_31, resumen_32, resumen_33], keys=['1', '2', '3'], names = ['ruta','vehicle'])

Suma de distancias y duración

In [199]:
res_general[3] = {'distancia': resumen_3.groupby(level = 'ruta').sum().sum()[0], 'duracion':resumen_3.groupby(level = 'ruta').sum().sum()[1], 'dur total':resumen_3['tiempo_total (hrs)'].sum()}

### K = 4

In [200]:
#Datos respecto al cluster en que se encuentran
X_41 = X_4[X_4.Cluster == 0]
X_42 = X_4[X_4.Cluster == 1]
X_43 = X_4[X_4.Cluster == 2]
X_44 = X_4[X_4.Cluster == 3]

In [201]:
# Ruta 1
mapa_41, resumen_41, recorrido_41 = ruta(X_41)

In [202]:
# Ruta 2
mapa_42, resumen_42, recorrido_42 = ruta(X_42)

In [203]:
# Ruta 3
mapa_43, resumen_43, recorrido_43 = ruta(X_43)

In [204]:
# Ruta 4
mapa_44, resumen_44, recorrido_44 = ruta(X_44)

Duración total de la ruta (con tiempo de espera)

In [205]:
resumen_41['tiempo_total (hrs)'] = recorrido_41.Arrival[recorrido_41.shape[0]-1] - recorrido_41.Arrival[0]
resumen_42['tiempo_total (hrs)'] = recorrido_42.Arrival[recorrido_42.shape[0]-1] - recorrido_42.Arrival[0]
resumen_43['tiempo_total (hrs)'] = recorrido_43.Arrival[recorrido_43.shape[0]-1] - recorrido_43.Arrival[0]
resumen_44['tiempo_total (hrs)'] = recorrido_44.Arrival[recorrido_44.shape[0]-1] - recorrido_44.Arrival[0]

Distancia, paquetes entregados y duración de cada ruta

In [206]:
resumen_4 = pd.concat([resumen_41, resumen_42, resumen_43, resumen_44], keys=['1', '2', '3','4'], names = ['ruta','vehicle'])

Suma de distancias y duración

In [207]:
res_general[4] = {'distancia': resumen_4.groupby(level = 'ruta').sum().sum()[0], 'duracion':resumen_4.groupby(level = 'ruta').sum().sum()[1], 'dur total':resumen_4['tiempo_total (hrs)'].sum()}

### K = 5

In [208]:
#Datos respecto al cluster en que se encuentran
X_51 = X_5[X_5.Cluster == 0]
X_52 = X_5[X_5.Cluster == 1]
X_53 = X_5[X_5.Cluster == 2]
X_54 = X_5[X_5.Cluster == 3]
X_55 = X_5[X_5.Cluster == 4]

In [209]:
# Ruta 1
mapa_51, resumen_51, recorrido_51 = ruta(X_51)

In [210]:
# Ruta 2
mapa_52, resumen_52, recorrido_52 = ruta(X_52)

In [211]:
# Ruta 3
mapa_53, resumen_53, recorrido_53 = ruta(X_53)

In [212]:
# Ruta 4
mapa_54, resumen_54, recorrido_54 = ruta(X_54)

In [213]:
# Ruta 5
mapa_55, resumen_55, recorrido_55 = ruta(X_55)

Duración total de la ruta (con tiempo de espera)

In [214]:
resumen_51['tiempo_total (hrs)'] = recorrido_51.Arrival[recorrido_51.shape[0]-1] - recorrido_51.Arrival[0]
resumen_52['tiempo_total (hrs)'] = recorrido_52.Arrival[recorrido_52.shape[0]-1] - recorrido_52.Arrival[0]
resumen_53['tiempo_total (hrs)'] = recorrido_53.Arrival[recorrido_53.shape[0]-1] - recorrido_53.Arrival[0]
resumen_54['tiempo_total (hrs)'] = recorrido_54.Arrival[recorrido_54.shape[0]-1] - recorrido_54.Arrival[0]
resumen_55['tiempo_total (hrs)'] = recorrido_55.Arrival[recorrido_55.shape[0]-1] - recorrido_55.Arrival[0]

Distancia, paquetes entregados y duración de cada ruta

In [215]:
resumen_5 = pd.concat([resumen_51, resumen_52, resumen_53, resumen_54, resumen_55], keys=['1', '2', '3','4','5'], names = ['ruta','vehicle'])

Suma de distancias y duración

In [216]:
res_general[5] = {'distancia': resumen_5.groupby(level = 'ruta').sum().sum()[0], 'duracion':resumen_5.groupby(level = 'ruta').sum().sum()[1], 'dur total':resumen_5['tiempo_total (hrs)'].sum()}

### K = 6

In [217]:
#Datos respecto al cluster en que se encuentran
X_61 = X_6[X_6.Cluster == 0]
X_62 = X_6[X_6.Cluster == 1]
X_63 = X_6[X_6.Cluster == 2]
X_64 = X_6[X_6.Cluster == 3]
X_65 = X_6[X_6.Cluster == 4]
X_66 = X_6[X_6.Cluster == 4]

In [218]:
# Ruta 1
mapa_61, resumen_61, recorrido_61 = ruta(X_61)

In [219]:
# Ruta 2
mapa_62, resumen_62, recorrido_62 = ruta(X_62)

In [220]:
# Ruta 3
mapa_63, resumen_63, recorrido_63 = ruta(X_63)

In [221]:
# Ruta 4
mapa_64, resumen_64, recorrido_64 = ruta(X_64)

In [222]:
# Ruta 5
mapa_65, resumen_65, recorrido_65 = ruta(X_65)

In [223]:
# Ruta 6
mapa_66, resumen_66, recorrido_66 = ruta(X_66)

Duración total de la ruta (con tiempo de espera)

In [224]:
resumen_61['tiempo_total (hrs)'] = recorrido_61.Arrival[recorrido_61.shape[0]-1] - recorrido_61.Arrival[0]
resumen_62['tiempo_total (hrs)'] = recorrido_62.Arrival[recorrido_62.shape[0]-1] - recorrido_62.Arrival[0]
resumen_63['tiempo_total (hrs)'] = recorrido_63.Arrival[recorrido_63.shape[0]-1] - recorrido_63.Arrival[0]
resumen_64['tiempo_total (hrs)'] = recorrido_64.Arrival[recorrido_64.shape[0]-1] - recorrido_64.Arrival[0]
resumen_65['tiempo_total (hrs)'] = recorrido_65.Arrival[recorrido_65.shape[0]-1] - recorrido_65.Arrival[0]
resumen_66['tiempo_total (hrs)'] = recorrido_66.Arrival[recorrido_65.shape[0]-1] - recorrido_65.Arrival[0]

Distancia, paquetes entregados y duración de cada ruta

In [225]:
resumen_6 = pd.concat([resumen_61, resumen_62, resumen_63, resumen_64, resumen_65, resumen_66], keys=['1', '2', '3','4','5','6'], names = ['ruta','vehicle'])

Suma de distancias y duración

In [226]:
res_general[6] = {'distancia': resumen_6.groupby(level = 'ruta').sum().sum()[0], 'duracion':resumen_6.groupby(level = 'ruta').sum().sum()[1],'dur total':resumen_6['tiempo_total (hrs)'].sum()}

## Resultados generales

In [227]:
pd.DataFrame(res_general)

,1,2,3,4,5,6
distancia,137063,192841,193579,262805,304905,367404
duracion,14039,17916,18125,22959,26286,31113
dur total,0 days 11:23:59,0 days 12:28:36,0 days 12:32:05,0 days 13:52:39,0 days 14:48:06,0 days 15:58:33
